# Meetup type: training of a classifier

Approach:
- Use dataset produced by `MeetupType_prototypeSentences`

Problems:
- Build a balanced training set
- Compute F1 for each class not working at the moment
- Repeat with larger set of prototype sentences (use 2 = 14447 items) - Done
- Find metric to measure probabilistic predictions
- Validate test set against all classes

In [1]:
!pip install sklearn
!pip install sentence_transformers
!pip install pickle
!pip install pandas
!pip install numpy

You should consider upgrading via the '/Users/ed4565/Development/polifonia/meetups_pilot/.python3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/ed4565/Development/polifonia/meetups_pilot/.python3/bin/python3 -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
You should consider upgrading via the '/Users/ed4565/Development/polifonia/meetups_pilot/.python3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/ed4565/Development/polifonia/meetups_pilot/.python3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/ed4565/Development/polifonia/meetups_pilot/.python3/bin/python3 -m pip install --upgrade pip' command.


In [2]:
inputIndex = 2
inputFile = 'meetupType/prototypeSentences_' + str(inputIndex) + '.csv'
save_path = 'meetupType/models'
save_models = True
repeated_sample = False
#emb_type = 'sbert' # sbert,

In [3]:
# Imports
from sklearn.neural_network import MLPClassifier
import sklearn.metrics
from sklearn import tree
from sklearn import linear_model
from sklearn.model_selection import train_test_split



In [4]:
import csv
# XXX Work in progress
# Build a balanced training set
# sentences,sentenceIndex,paragraphIndex,section,file,wikiId,

labels = ['Music making','Business meeting','Personal life','Coincidence','Public celebration','Education' ]            
sentences_map = {}
with open(inputFile) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        first = True
        for row in csv_reader:
            if first:
                first = False
                continue
            s = row[0] # 6-12
            for i in range(6,12):
                if row[i] == 'True': # 
                    l = labels[i-6]
                    if l not in sentences_map.keys():
                        sentences_map[l] = []
                    sentences_map[l].append(s)

counts = {}
for key in sentences_map.keys():
    counts[key] = len(sentences_map[key])

# Build the training set by selecting first sentences from low represented classes
counts={k: v for k, v in sorted(counts.items(), key=lambda item: item[1])}
for key in counts.keys():
    print(key,counts[key])

# TODO Build balanced training set
items_per_class = next(iter(counts.values()))
print('Producing ' + str(items_per_class) + ' items per class')
#for key in counts.keys():


Education 629
Coincidence 952
Public celebration 1200
Business meeting 2113
Personal life 4074
Music making 7172
Producing 629 items per class


In [5]:
import csv
# OLD method to build the train_test set
# sentences,sentenceIndex,paragraphIndex,section,file,wikiId,

labels = ['Music making','Business meeting','Personal life','Coincidence','Public celebration','Education' ]            
sentences = []
types = []
count_added = {}
for key in counts.keys():
    count_added[key] = 0

with open(inputFile) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        first = True
        for row in csv_reader:
            if first:
                first = False
                continue
            s = row[0] # 6-11
            for i in range(6,12):
                if row[i] == 'True' and count_added[labels[i-6]] <= items_per_class: # 
                    sentences.append(s)
                    types.append(labels[i-6])
                    count_added[labels[i-6]] = 1 + count_added[labels[i-6]]
                    if not repeated_sample:
                        break

list(zip(sentences, types))[:5]       

[('He is best known for writing the Hymn to Liberty (Greek: Ὕμνος εις την Ἐλευθερίαν, Ýmnos eis tīn Eleutherían), which was set to music by Nikolaos Mantzaros and became the Greek and Cypriot national anthem in 1865 and 1966 respectively.',
  'Music making'),
 ('He was the central figure of the Heptanese School of poetry, and is considered the national poet of Greece—not only because he wrote the national anthem, but also because he contributed to the preservation of earlier poetic tradition and highlighted its usefulness to modern literature.',
  'Music making'),
 ('Born in Zakynthos in 1798, Dionysios Solomos was the illegitimate child of a wealthy count, Nikolaos Solomos, and his housekeeper, Angeliki Nikli.',
  'Personal life'),
 ("Nikolaos Solomos was of Cretan origin; his family were Cretan refugees who settled on Zakynthos in 1670 after Crete's conquest by the Ottoman Empire in 1669.",
  'Personal life'),
 ('It is possible that his mother Angeliki Nikli came from the region of M

In [6]:
import collections
counter=collections.Counter(types)
print(counter)

Counter({'Music making': 630, 'Personal life': 630, 'Coincidence': 630, 'Public celebration': 630, 'Business meeting': 630, 'Education': 581})


In [7]:
from sklearn.model_selection import train_test_split
print(len(sentences), len(types))
S_train, S_test, y_train, y_test = train_test_split(sentences, types, train_size = 0.8, stratify = types)
#list(zip(X, y))[:10]
print(S_train[:10],y_train[:10])
# 1 - lentgh 2651 2651

3731 3731
['Romero died three days later leading to reports questioning if the film will ever be made.', 'In 1961, she made her Broadway  debut in How to Succeed in Business Without Really Trying, where she met choreographer Bob Fosse and his wife, Gwen Verdon.', 'He co-wrote the song "It\'s the Blues (No. 14 Blues)" which was recorded in Detroit and released by Victor.', "Euday Bowman's work is not public domain.", 'Muhal Richard Abrams (born Richard Lewis Abrams; September 19, 1930 – October 29, 2017) was an American educator, administrator, composer, arranger, clarinetist, cellist, and jazz pianist in the free jazz medium.', 'Sheinberg is credited with discovering director Steven Spielberg.', 'Die Schachtel, Zeit C01Archives sauvées des eaux – Luc Ferrari with Otomo Yoshihide.', "A talented musician, Heath won the college's organ scholarship in his first term (he had previously tried for the organ scholarships at St Catharine's College, Cambridge, and Keble College, Oxford) which en

In [8]:
import time
# Using the seed sentences as training set with a sentencebert based classifier
# https://medium.com/dair-ai/tl-dr-sentencebert-8dec326daf4e
# Models https://www.sbert.net/docs/pretrained_models.html
# SBERT
from sentence_transformers import SentenceTransformer 
t0 = time.monotonic()
model = SentenceTransformer('all-mpnet-base-v2')
t1 = time.monotonic()
print('Model loaded in ', t1 - t0)
#Sentences are encoded by calling model.encode() 
X_train = model.encode(S_train)
t2 = time.monotonic()
print('Train embeddings produced in ', t2 - t1)
X_test = model.encode(S_test)
t3 = time.monotonic()
print('Train embeddings produced in ', t3 - t2)


Model loaded in  7.749250255999982
Train embeddings produced in  233.052718615
Train embeddings produced in  58.33115924499998


In [9]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)



In [25]:
results = []
import sklearn
import pickle
# Precision, Recall, F1 and Matthews' Correlation
# sklearn.metrics.matthews_corrcoef(y_true, y_pred, *, sample_weight=None)
def getAccuracy(y_test, predictions):
    return sklearn.metrics.accuracy_score(y_test, predictions, normalize=True, sample_weight=None)

def getF1(y_test, predictions):
    f1 = []
    for l in sorted(set(y_test)):
        print("F1 for label ",l)
        f1.append(sklearn.metrics.f1_score(y_test, predictions, labels=[l], average=None)[0])
        #f1.append(sklearn.metrics.f1_score(y_test, predictions, average='macro'))
    return f1

def getMatthews(y_test,predictions):
    return sklearn.metrics.matthews_corrcoef(y_test,predictions)


def save_model(clf, path):
    fname = type(clf).__name__ + '_' + str(inputIndex) + '.clf'
    pickle.dump(clf, open(path + '/' + fname, 'wb'))

In [26]:
# MLPClassifier
from sklearn.neural_network import MLPClassifier
#print(y_test)
clf = MLPClassifier(random_state=1, max_iter=600)
clf.fit(X_train, y_train)
MLPClassifierPredictions = clf.predict(X_test)
MLPClassifierAccuracy = getAccuracy(y_test, MLPClassifierPredictions)
MLPClassifierF1 = getF1(y_test, MLPClassifierPredictions)

MLPClassifierMatthews = getMatthews(y_test, MLPClassifierPredictions)
results.append(["MLPClassifier", MLPClassifierAccuracy, MLPClassifierMatthews]+MLPClassifierF1)
if(save_models):
    save_model(clf, save_path)

F1 for label  Business meeting
F1 for label  Coincidence
F1 for label  Education
F1 for label  Music making
F1 for label  Personal life
F1 for label  Public celebration


In [27]:
import pandas as pd
print(pd.DataFrame(results, columns=["Approach", "Accuracy", "Matthews"]+sorted(set(y_test))))

# PAST RESULTS
# Approach  Accuracy  Matthews  Business meeting  Coincidence  \
# 0  MLPClassifier   0.63606  0.464063          0.493197     0.493197   

#    Education  Music making  Personal life  Public celebration  
# 0   0.493197      0.493197       0.493197            0.493197

        Approach  Accuracy  Matthews  Business meeting  Coincidence  \
0  MLPClassifier     0.707     0.648             0.724        0.675   

   Education  Music making  Personal life  Public celebration  
0      0.762         0.628          0.744               0.707  


## Note
The test results include the labels in a different order than the one in the training set (they are sorted).
The classifier generates scores considering the labels as ordered in the training set file:



In [31]:
import numpy
proba = clf.predict_proba(X_test)
#print(zip(proba[:10]))
output_proba = []
for index, item in enumerate(S_test):
    #print(ix[0][0],proba[index])
    #print([sentences[int(ix[0][0])]], proba[index])
    #print(item,y_test[index])
    output_proba.append([item, y_test[index]] + list(proba[index]))
    #break

#print(senten1)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.DataFrame(output_proba, columns=['Sentence','Test_class']+sorted(labels))
print(df[:10])
df.to_csv(save_path + '/MLPClassifier_test_results_' + str(inputIndex) + '.csv', float_format = '%.6f')

                                            Sentence          Test_class  \
0  Goldsmith was honoured with the 15th Music Ind...  Public celebration   
1  In 1940, Vian met Michelle Léglise, who became...         Coincidence   
2  He switches ragas with ease, and sings complex...        Music making   
3  By 1944, Skelton was engaged to actress Muriel...       Personal life   
4  The re-recording peaked at No. 9 one week to t...        Music making   
5  1968: Peter Thomas, Ika Schafheitlin, Helmuth ...       Personal life   
6  John Herbert Gleason was born on February 26, ...       Personal life   
7  Max had also been commissioned to design bilin...    Business meeting   
8                   He has been married three times.       Personal life   
9  During the 1960s, as the public taste shifted ...         Coincidence   

   Business meeting  Coincidence  Education  Music making  Personal life  \
0             0.004        0.000      0.000         0.000          0.000   
1          